In [7]:
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
import numpy as np
from sklearn.model_selection import train_test_split
from detect_outlier import *


In [8]:
df_normalized = read_data()
df_normalized.dropna(inplace=True)


In [9]:
df_normalized.describe()


,numberOfLiquidation,totalValueOfLiquidation,frequencyOfDappTransactions,numberOfInteractedDapps,typesOfInteractedDapps,numberOfReputableDapps,frequencyOfTransaction,totalAsset,frequencyMountOfTransaction,1st_label,2nd_label,borrow_per_balance,borrow_per_deposit,averageTotalAsset,deposit_per_asset,age
count,114338.000000,114338.000000,114338.000000,114338.000000,114338.000000,114338.000000,114338.000000,114338.000000,114338.000000,114338.000000,114338.000000,114338.000000,114338.000000,114338.000000,114338.000000,114338.000000
mean,848.919126,849.399605,337.791154,332.870087,324.661717,323.235349,310.000269,336.086648,314.754811,1.201849,1.337735,765.043513,800.889564,335.609621,678.708908,515.909421
std,15.891089,12.248816,116.639385,104.624034,87.740470,90.163433,48.799789,75.777304,18.118774,0.554510,0.781509,194.900810,126.669295,75.263585,209.897732,133.112589
min,300.000000,303.499740,300.000000,300.000000,300.000000,300.000000,300.000000,300.000010,300.000329,0.000000,0.000000,300.000001,300.000000,300.000036,300.000062,300.000000
25%,850.000000,850.000000,300.000000,300.000000,300.000000,300.000000,300.000000,302.418960,313.674664,1.000000,1.000000,850.000000,850.000000,302.548531,466.480961,436.046160
50%,850.000000,850.000000,300.000000,300.000000,300.000000,300.000000,300.000000,313.390772,313.674664,1.000000,1.000000,850.000000,850.000000,312.694751,822.155832,509.713504
75%,850.000000,850.000000,300.000000,300.000000,300.000000,300.000000,300.000000,326.463743,313.674664,1.000000,1.000000,850.000000,850.000000,326.034284,850.000000,547.023778
max,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,849.976466,849.872010,4.000000,4.000000,850.000000,850.000000,849.943804,850.000000,850.000000


In [10]:
X = np.array(df_normalized[
    [
        "totalAsset",
        "averageTotalAsset",
        "frequencyMountOfTransaction",
        "borrow_per_balance",
        "deposit_per_asset",
        "borrow_per_deposit",
        "totalValueOfLiquidation",
        "numberOfLiquidation",
        "frequencyOfTransaction",
        "frequencyOfDappTransactions",
        "numberOfInteractedDapps",
        "typesOfInteractedDapps",
        "numberOfReputableDapps",
        "age",
    ]
].values)
main_y = df_normalized["1st_label"].values
sub_y = df_normalized["2nd_label"].values
# y = np.array([score(y[i]) for i in range(len(y))])


In [11]:
num_nan = np.sum(np.isnan(X))
print("Số lượng phần tử NaN trong danh sách là:", num_nan)

Số lượng phần tử NaN trong danh sách là: 0


In [12]:
import numpy as np
def nomal(arr):
    sum = np.sum(arr)
    arr = arr / sum
    return arr

In [13]:
import random
import numpy as np
from sklearn.model_selection import train_test_split

class Genetic_Alogorithm:
    def __init__(
        self, X, main_y, sub_y, num_params, num_solutions=100, mutation_rate=0.2
    ):
        self.X = X
        self.main_y = main_y
        self.sub_y = sub_y
        self.num_params = num_params
        self.num_solutions = num_solutions
        self.mutation_rate = mutation_rate
        (
            self.X_train,
            self.X_test,
            self.y_train,
            self.y_test,
            self.sub_y_train,
            self.sub_y_test,
        ) = train_test_split(
            X, main_y, sub_y, test_size=0.2, stratify=main_y, random_state=42
        )

    def initialize_solutions(self):
        solutions = []
        for _ in range(self.num_solutions):
            # Sử dụng np.random.uniform để khởi tạo giá trị trong khoảng từ -500 đến 500
            solution = np.random.uniform(0, 1, size=self.num_params)
            solutions.append(tuple(nomal(solution)))
        return solutions

    def mutate(self, child, mutation_rate):
        if mutation_rate is None:
            mutation_rate = self.mutation_rate
        mutated_child = tuple(
            [
                gene * np.random.uniform(1 - self.mutation_rate, 1 + self.mutation_rate)
                for gene in child
            ]
        )
        return mutated_child

    def crossover(self, parent1, parent2, type=None):
        if type is None or type == "one_point":
            crossover_point = np.random.randint(1, len(parent1) - 1)
            child = list(parent1[:crossover_point]) + list(parent2[crossover_point:])
            return tuple(child)
        elif type == "two_point":
            crossover_point1, crossover_point2 = np.sort(
                np.random.choice(range(1, len(parent1)), 2, replace=False)
            )
            child = (
                list(parent1[:crossover_point1])
                + list(parent2[crossover_point1:crossover_point2])
                + list(parent1[crossover_point2:])
            )
            return tuple(nomal(child))
        elif type == "mean":
            child = [(parent1[i] + parent2[i]) / 2 for i in range(len(parent1))]
            return tuple(child)

    def evolve(self, solutions, crossover="None", mutate_rate="None"):
        rankedsolutions = [(self.fitness(theta), theta) for theta in solutions]
        rankedsolutions = sorted(rankedsolutions, key=lambda x: x[0], reverse=True)
        print(f"fitness:{rankedsolutions[0][0]}")
        bestSolutions = rankedsolutions[:20] + rankedsolutions[-5:]

        new_solution = [rankedsolutions[0][1]]

        for _ in range(self.num_solutions - 1):
            parent1, parent2 = (
                random.choice(bestSolutions)[1],
                random.choice(bestSolutions)[1],
            )
            child1 = self.crossover(parent1, parent2, crossover)
            child1 = self.mutate(child1, mutate_rate)
            new_solution.append(nomal(child1))
        return new_solution, rankedsolutions[0][0]

    # support funcion
    def fitness(self, theta):
        result = 0
        y_pred = self.predict(self.X_train, theta)
        return self.accuracy_score(self.y_train, self.sub_y_train, y_pred)
        for yp, yt in zip(y_pred, self.y_train):
            diff = abs(yt - yp)
            if diff == 0:
                continue
            elif diff == 1:
                result += 200
            elif diff == 2:
                result += 400
            elif diff == 3:
                result += 800
            else:
                result += 2000
        return result

    def accuracy_score(self, y_train, sub_y_train, y_pred):
        y_train = np.array(y_train)
        sub_y_train = np.array(sub_y_train)
        y_pred = np.array(y_pred)
        condition = np.logical_or(y_pred == sub_y_train, y_pred == y_train)
        count = np.sum(condition)
        accuracy = count / len(y_train)
        return accuracy

    def predict(self, matrices, theta):
        list_scores = np.dot(matrices, theta)
        label = []
        for score in list_scores:
            if  score < 580:
                label.append(0)
            elif score >= 580 and score < 670:
                label.append(1)
            elif score >= 670 and score < 740:
                label.append(2)
            elif score >= 740 and score < 800:
                label.append(3)
            elif 800<= score <= 850:
                label.append(4)
            else:
                print(score)
        return np.array(label)

In [14]:
ga = Genetic_Alogorithm(X, main_y, sub_y, len(X[0]))

In [15]:
solution = ga.initialize_solutions()
fitness = []
num_generations = 100
for _ in range(num_generations):
    solution, acc = ga.evolve(solution, crossover="two_point", mutate_rate=0.1)
    fitness.append(acc)

fitness:0.565649939870996
fitness:0.7339674210123538
fitness:0.7445938559090413
fitness:0.7825297911883677
fitness:0.7825297911883677
fitness:0.7878539411829015
fitness:0.8018257352137312
fitness:0.8065048649830545
fitness:0.8227287635290259
fitness:0.8227287635290259
fitness:0.845829233628512
fitness:0.8831310812288182
fitness:0.8935607302940854
fitness:0.8944025363507161
fitness:0.9049852410626434
fitness:0.9049852410626434
fitness:0.9049852410626434
fitness:0.9075215917787253
fitness:0.9093691920848366
fitness:0.9093691920848366
fitness:0.9093691920848366
fitness:0.9141139171312999
fitness:0.9148463977260304
fitness:0.9168798513173718
fitness:0.9190007652782333
fitness:0.9197660435115338
fitness:0.9217557669181152
fitness:0.9236252323166065
fitness:0.9236252323166065
fitness:0.9244998360118072
fitness:0.9244998360118072
fitness:0.9248824751284574
fitness:0.9248824751284574
fitness:0.9269596589045589
fitness:0.9269596589045589
fitness:0.9269596589045589
fitness:0.9276921394992894
fit

In [16]:
solution[0]

array([1.37762961e-01, 1.52787632e-01, 5.15937019e-02, 6.17449011e-04,
       4.74114916e-04, 3.66810350e-03, 5.85067434e-02, 5.81493842e-01,
       8.60370232e-04, 1.46675050e-03, 2.50422272e-03, 2.54036698e-03,
       2.41204277e-03, 3.31169979e-03])

In [17]:
pred = ga.predict(ga.X_test, solution[0])
unique_elements, counts = np.unique(pred, return_counts=True)
    
for element, count in zip(unique_elements, counts):
    print("Phần tử", element, "xuất hiện", count, "lần")


Phần tử 0 xuất hiện 52 lần
Phần tử 1 xuất hiện 19121 lần
Phần tử 2 xuất hiện 3070 lần
Phần tử 3 xuất hiện 588 lần
Phần tử 4 xuất hiện 37 lần
